In [1]:
import numpy as np
import pandas as pd
import argus_shapes

import sklearn.base as sklb
import sklearn.model_selection as sklms

/home/mbeyeler/anaconda3/lib/python3.5/site-packages/skvideo/__init__.py:356: UserWarning: avconv/avprobe not found in path: 
  warnings.warn("avconv/avprobe not found in path: " + str(path), UserWarning)
2018-04-23 17:25:37,922 [pulse2percept] [INFO] Welcome to pulse2percept


In [2]:
class DummyRegressor(sklb.BaseEstimator, sklb.RegressorMixin):
    def __init__(self, a=1):
        self.a = a
        self.greater_is_better = True
        
    def fit(self, X, y=None, **fit_params):
        return self
    
    def predict(self, X):
        y_pred = []
        for i, _ in X.iterrows():
            y_pred.append(i * self.a)
        return np.array(y_pred)

In [3]:
dummy = DummyRegressor(a=0)

In [22]:
search_params = {'a': (-100, 100)}
pso = argus_shapes.model_selection.ParticleSwarmOptimizer(dummy, search_params, min_func=0.00001, verbose=False)
print(pso)

ParticleSwarmOptimizer(estimator=DummyRegressor(a=0.98329713053353363),
            max_iter=100, min_func=1e-05, min_step=0.01,
            search_params={'a': (-100, 100)}, swarm_size=10, verbose=False)


In [23]:
n_samples = 12
X = pd.DataFrame(np.repeat(np.arange(n_samples), 2).reshape((-1, 2)), columns=['feat1', 'feat2'])
y = pd.DataFrame(np.arange(n_samples), columns=['target'])

In [24]:
pso.fit(X, y=y)

Stopping search: Swarm best position change less than 0.01
Best err: -0.999998220008 Best params: {'a': 1.0007092539290121}


In [25]:
pso.score(X, y)

0.99999822000828875

In [26]:
y_test, y_pred, best_params, train_score, test_score = argus_shapes.model_selection.crossval_predict(
    pso, X, y, n_folds=5
)

Fold 1 / 5
Stopping search: Swarm best objective change less than 1e-05
Best err: -0.999997808597 Best params: {'a': 0.99948770757409167}
Fold 2 / 5
Stopping search: Swarm best objective change less than 1e-05
Best err: -0.999999343715 Best params: {'a': 0.99956413620309859}
Fold 3 / 5
Stopping search: Swarm best position change less than 0.01
Best err: 1.77987144289 Best params: {'a': 0.038187591254681408}
Fold 4 / 5
Stopping search: Swarm best position change less than 0.01
Best err: -0.998202978096 Best params: {'a': 0.97546781478931099}
Fold 5 / 5
Stopping search: Swarm best position change less than 0.01
Best err: -0.999949187663 Best params: {'a': 1.0038352098338652}


In [27]:
argus_shapes.model_selection.crossval_score(y_test, y_pred, metric='r2')

[0.99999934389117584,
 0.99999525056876382,
 -156.26412863480022,
 0.82546984774865151,
 0.99349869516435885]